In [ ]:
from __future__ import absolute_import, division, print_function

from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import os

from tensorflow.keras import layers
from tensorflow.keras import optimizers

import pandas as pd
import numpy as np

tf.VERSION

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = image_generator.flow_from_directory("flower_photos")
validation_data = image_generator.flow_from_directory("flower_photos_test")

for image_batch,label_batch in train_data:
  print("Image batch shape: ", image_batch.shape)
  print("Labe batch shape: ", label_batch.shape)
  break

class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []
    self.epoch_val_loss = []
    self.epoch_val_acc = []
    
  def on_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
  def on_epoch_end(self, epoch, logs=None):
    self.epoch_val_loss.append(logs['val_loss'])
    self.epoch_val_acc.append(logs['val_acc'])

runs = [2, 3]
activation = [0, 0, 1, 1, 2, 2]
lossF = ['categorical_crossentropy', 'mean_squared_error', 'categorical_crossentropy', 'mean_squared_error', 'categorical_crossentropy', 'mean_squared_error']
saveName = ['Tanh', 'MSETanh', 'RELU', 'MSERELU', 'LeakyRELU', 'MSELeakyRELU']

for id in runs:
    model = None
    if (activation[id] == 0):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    if (activation[id] == 1):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    if (activation[id] == 2):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    model.compile(
        optimizer=tf.train.AdamOptimizer(), 
        loss=lossF[id],
        metrics=['accuracy'])
    
    batch_stats = CollectBatchStats()
    model.fit_generator(train_data, epochs=4, 
        callbacks = [batch_stats],
        validation_data = validation_data)
    
    results = pd.DataFrame({"losses":batch_stats.batch_losses, "accuracy":batch_stats.batch_acc})
    results.to_csv("saved_results/cnn"+saveName[id]+"Data.csv", ',')
    results = pd.DataFrame({"losses":batch_stats.epoch_val_loss, "accuracy":batch_stats.epoch_val_acc})
    results.to_csv("saved_results/cnn"+saveName[id]+"ValData.csv", ',')

W0515 11:34:02.326097 140061927892800 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Image batch shape:  (32, 256, 256, 3)
Labe batch shape:  (32, 5)
Instructions for updating:
Colocations handled automatically by placer.


W0515 11:34:03.740329 140061927892800 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


W0515 11:34:03.943371 140061927892800 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/utils/losses_utils.py:170: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


W0515 11:34:04.120528 140061927892800 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/4
104/104 [==============================] - 787s 8s/step - loss: 0.3287 - acc: 0.1738 - val_loss: 0.3308 - val_acc: 0.1717
Epoch 2/4
104/104 [==============================] - 844s 8s/step - loss: 0.3306 - acc: 0.1726 - val_loss: 0.3320 - val_acc: 0.1717
Epoch 3/4
104/104 [==============================] - 762s 7s/step - loss: 0.3315 - acc: 0.1726 - val_loss: 0.3320 - val_acc: 0.1717
Epoch 4/4
104/104 [==============================] - 829s 8s/step - loss: 0.3311 - acc: 0.1726 - val_loss: 0.3308 - val_acc: 0.1717
Epoch 1/4
104/104 [==============================] - 797s 8s/step - loss: 1.5099 - acc: 0.4329 - val_loss: 1.2531 - val_acc: 0.5204
Epoch 2/4
104/104 [==============================] - 802s 8s/step - loss: 0.9896 - acc: 0.6225 - val_loss: 1.2000 - val_acc: 0.5313
Epoch 3/4
104/104 [==============================] - 829s 8s/step - loss: 0.6959 - acc: 0.7466 - val_loss: 1.2233 - val_acc: 0.5831
Epoch 4/4
104/104 [==============================] - 853s 8s/step - loss: 0.

In [1]:
results = pd.DataFrame({"losses":batch_stats.batch_losses, "accuracy":batch_stats.batch_acc})
results.to_csv("saved_results/cnn"+saveName[id]+"Data.csv", ',')
results = pd.DataFrame({"losses":batch_stats.epoch_val_loss, "accuracy":batch_stats.epoch_val_acc})
results.to_csv("saved_results/cnn"+saveName[id]+"ValData.csv", ',')

NameError: name 'pd' is not defined